In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, mean
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# 1. Khởi tạo SparkSession
spark = SparkSession.builder.appName("ProvinceNextYearTempPrediction").getOrCreate()


In [ ]:
# 2. Đọc dữ liệu
file_path = "/content/sorted_weather.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)


In [ ]:
# 3. Chuẩn bị dữ liệu
# Thêm cột "year"
data = data.withColumn("year", year(col("date")))

In [ ]:
# Tính nhiệt độ trung bình
data = data.withColumn("avg_temp", (col("max") + col("min")) / 2)

In [ ]:
# Chọn các cột đặc trưng
feature_columns = ['max', 'min', 'wind', 'rain', 'humidi', 'cloud', 'pressure']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

# Chọn các cột cần thiết
data = data.select("province", "year", "features", "avg_temp")

# Lấy danh sách các tỉnh
provinces = data.select("province").distinct().rdd.flatMap(lambda x: x).collect()

# Lưu kết quả dự đoán
predictions_by_province = []

In [ ]:
# 4. Dự đoán cho từng tỉnh
for province in provinces:
    print(f"Processing province: {province}")

    # Lọc dữ liệu cho từng tỉnh
    province_data = data.filter(data["province"] == province)

    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    train_data = province_data.filter(col("year") < 2018)
    test_data = province_data.filter(col("year") == 2018)

    # Kiểm tra đủ dữ liệu để huấn luyện
    if train_data.count() > 0 and test_data.count() > 0:
        # Huấn luyện mô hình
        rf = RandomForestRegressor(featuresCol="features", labelCol="avg_temp", numTrees=100, seed=42)
        rf_model = rf.fit(train_data)

        # Dự đoán trên tập kiểm tra
        predictions = rf_model.transform(test_data)
        evaluator = RegressionEvaluator(labelCol="avg_temp", predictionCol="prediction", metricName="rmse")
        rmse = evaluator.evaluate(predictions)
        print(f"RMSE for province {province}: {rmse:.2f}")

        # Dự đoán nhiệt độ trung bình năm sau
        next_year_data = test_data.withColumn("year", col("year") + 1)  # Giả định dữ liệu của năm tiếp theo
        next_year_predictions = rf_model.transform(next_year_data)

        # Tính nhiệt độ trung bình năm sau
        avg_temp_2019 = next_year_predictions.select(mean("prediction").alias("avg_temp_2019")).collect()[0]["avg_temp_2019"]
        predictions_by_province.append((province, avg_temp_2019))
    else:
        print(f"Not enough data for province {province}. Skipping.")

# 5. Tạo DataFrame kết quả
result_df = spark.createDataFrame(predictions_by_province, ["province", "avg_temp_2019"])

# Hiển thị kết quả
result_df.show()



Processing province: Long Xuyen
RMSE for province Long Xuyen: 0.22
Processing province: My Tho
RMSE for province My Tho: 0.21
Processing province: Ho Chi Minh City
RMSE for province Ho Chi Minh City: 0.19
Processing province: Ben Tre
RMSE for province Ben Tre: 0.21
Processing province: Nha Trang
RMSE for province Nha Trang: 0.21
Processing province: Hong Gai
RMSE for province Hong Gai: 0.37
Processing province: Hai Phong
RMSE for province Hai Phong: 0.41
Processing province: Cam Pha
RMSE for province Cam Pha: 0.32
Processing province: Hai Duong
RMSE for province Hai Duong: 0.40
Processing province: Ca Mau
RMSE for province Ca Mau: 0.18
Processing province: Cam Ranh
RMSE for province Cam Ranh: 0.20
Processing province: Play Cu
RMSE for province Play Cu: 0.49
Processing province: Nam Dinh
RMSE for province Nam Dinh: 0.42
Processing province: Chau Doc
RMSE for province Chau Doc: 0.23
Processing province: Can Tho
RMSE for province Can Tho: 0.18
Processing province: Da Lat
RMSE for province

In [ ]:
# Lấy 5 tỉnh có nhiệt độ trung bình cao nhất
top_5_hottest = result_df.orderBy(col("avg_temp_2019").desc()).limit(5)
print("5 tỉnh có nhiệt độ trung bình cao nhất:")
top_5_hottest.show()

# Lấy 5 tỉnh có nhiệt độ trung bình thấp nhất
top_5_coldest = result_df.orderBy(col("avg_temp_2019").asc()).limit(5)
print("5 tỉnh có nhiệt độ trung bình thấp nhất:")
top_5_coldest.show()

5 tỉnh có nhiệt độ trung bình cao nhất:
+----------------+------------------+
|        province|     avg_temp_2019|
+----------------+------------------+
|          Tan An| 29.38790324253332|
|        Bien Hoa|29.385421403556926|
|Ho Chi Minh City|29.385411228000745|
|Ho Chi Minh City|29.385411228000745|
|        Chau Doc|29.212288550315748|
+----------------+------------------+

5 tỉnh có nhiệt độ trung bình thấp nhất:
+--------+------------------+
|province|     avg_temp_2019|
+--------+------------------+
|  Da Lat|  21.0727885880052|
| Cam Pha|23.929775082735606|
| Cam Pha|23.929775082735606|
| Play Cu| 24.05479151636748|
| Uong Bi|24.055729008541267|
+--------+------------------+

